In [3]:
from docplex.mp.model import Model

m = Model(name='telephone_production')

In [4]:
desk = m.continuous_var(name='desk')
cell = m.continuous_var(name='cell')

In [5]:
m.add_constraint(desk >= 100)

m.add_constraint(cell >= 100)

ct_assembly = m.add_constraint(0.2 * desk + 0.4 * cell <= 400)

ct_painting = m.add_constraint(0.5 * desk + 0.4 * cell <= 490)

In [6]:
m.maximize(12 * desk + 20 * cell)

In [7]:
m.print_information()

Model: telephone_production
 - number of variables: 2
   - binary=0, integer=0, continuous=2
 - number of constraints: 4
   - linear=4
 - parameters: defaults
 - objective: maximize
 - problem type is: LP


In [8]:
s = m.solve()

In [9]:
m.print_solution()

objective: 20600.000
status: OPTIMAL_SOLUTION(2)
  desk=300.000
  cell=850.000


In [10]:
# create a new model, copy of m
im = m.copy()
# get the 'desk' variable of the new model from its name
idesk = im.get_var_by_name('desk')
# add a new (infeasible) constraint
im.add_constraint(idesk >= 1100);
# solve the new proble, we expect a result of None as the model is now infeasible
ims = im.solve()
if ims is None:
    print('- model is infeasible')

-- cannot find parameters matching version: 22.1.2.0, using: 22.1.1.0
- model is infeasible


In [11]:
# sof constaints
overtime = m.continuous_var(name='overtime', ub=40)
ct_assembly.rhs = 400 + overtime

m.maximize(12 * desk + 20 * cell - 2 * overtime)

In [12]:
s2 = m.solve()
m.print_solution()

objective: 22253.333
status: OPTIMAL_SOLUTION(2)
  desk=166.667
  cell=1016.667
  overtime=40.000


In [13]:
print('* desk variable has reduced cost: {0}'.format(desk.reduced_cost))
print('* cell variable has reduced cost: {0}'.format(cell.reduced_cost))

* desk variable has reduced cost: 0
* cell variable has reduced cost: 0


In [16]:
# revert soft constraints
ct_assembly.rhs = 400
s3 = m.solve()

# now get slack value for assembly constraint: expected value is 40
print('* slack value for assembly time constraint is: {0}'.format(ct_assembly.slack_value))
# get slack value for painting time constraint, expected value is 0.
print('* slack value for painting time constraint is: {0}'.format(ct_painting.slack_value))

* slack value for assembly time constraint is: 0
* slack value for painting time constraint is: 0


In [17]:
m.parameters.lpmethod = 4
m.solve(log_output=True)

Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
CPXPARAM_LPMethod                                4
Tried aggregator 1 time.
LP Presolve eliminated 2 rows and 1 columns.
Reduced LP has 2 rows, 2 columns, and 4 nonzeros.
Presolve time = 0.15 sec. (0.00 ticks)
Parallel mode: using up to 32 threads for barrier.
Number of nonzeros in lower triangle of A*A' = 1
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.02 sec. (0.00 ticks)
Summary statistics for Cholesky factor:
  Threads                   = 32
  Rows in Factor            = 2
  Integer space required    = 2
  Total non-zeros in factor = 3
  Total FP ops to factor    = 5
 Itn      Primal Obj        Dual Obj  Prim Inf Upper Inf  Dual Inf Inf Ratio
   0   8.9784782e+03  -3.2000000e+03  5.68e-14  0.00e+00  6.60e+01  1.00e+00
   1   1.7820775e+04   9.6586492e+03  0.00e+00  0.00e+00  1.84e+01  8.19e-01
   2   1.7373513e+04   1.8991282e+04  6.63e-11  0.00

docplex.mp.solution.SolveSolution(obj=20600,values={desk:300,cell:850})